#  App para autos usados Rusty Bargain 

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from pathlib import Path
import lightgbm as lgb
import timeit

In [2]:
#Revisión externa:

df = pd.read_csv('/datasets/car_data.csv')

#Revisión interna:

#df = pd.read_csv('car_data.csv')

In [3]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

En una primera observación de los datos podemos notar que hay datos ausentes en varias columnas, tipos de datos en formatos erroneos y una escritura incorrecta de los títulos de las columnas. 

## Preparación de datos

In [5]:
#Mejora de los nombres de cada columna.

df = df.rename(columns = {'DateCrawled': 'date_crawled', 'VehicleType': 'vehicle_type',
                          'RegistrationYear': 'registration_year','RegistrationMonth': 'registration_month',
                          'FuelType': 'fuel_type', 'NotRepaired': 'not_repaired', 'DateCreated': 'date_created',
                          'NumberOfPictures': 'number_pictures', 'PostalCode': 'postal_code', 'LastSeen': 'last_seen'})

In [6]:
#Minúsculas para todas las columnas.

df.columns = df.columns.str.lower()

In [7]:
#Cambio a formato tipo fecha de tres columnas.

df['date_crawled'] = pd.to_datetime(df['date_crawled'], format = '%d/%m/%Y %H:%M')

df['date_created'] = pd.to_datetime(df['date_created'], format = '%d/%m/%Y %H:%M')

df['last_seen'] = pd.to_datetime(df['last_seen'], format = '%d/%m/%Y %H:%M')

In [8]:
#Revisión de información duplicada.

df.duplicated().sum()

262

In [9]:
df[df.duplicated()].head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_pictures,postal_code,last_seen
14266,2016-03-21 19:06:00,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,2016-03-21,0,65529,2016-04-05 20:47:00
27568,2016-03-23 10:38:00,12200,bus,2011,manual,125,zafira,40000,10,gasoline,opel,no,2016-03-23,0,26629,2016-04-05 07:44:00
31599,2016-04-03 20:41:00,4950,wagon,2003,auto,170,e_klasse,150000,4,gasoline,mercedes_benz,no,2016-04-03,0,48432,2016-04-05 21:17:00
33138,2016-03-07 20:45:00,10900,convertible,2005,auto,163,clk,125000,5,petrol,mercedes_benz,no,2016-03-07,0,61200,2016-03-21 03:45:00
43656,2016-03-13 20:48:00,4200,sedan,2003,manual,105,golf,150000,10,gasoline,volkswagen,no,2016-03-13,0,14482,2016-03-13 20:48:00


In [10]:
#Se elimina la información duplicada. 

df = df.drop_duplicates().reset_index(drop = True)

df.duplicated().sum()

0

Se eliminarán varias columnas referentes a los datos personales de los usuarios, ya que estos no aportan en ninguna forma al desarrollo del modelo de Machine Learning. 

In [11]:
#Eliminación de la columna que no aporta al modelo de Machine Learning.

df = df.drop(['date_crawled', 'date_created', 'number_pictures', 'postal_code', 'last_seen'], axis=1)


In [12]:
df.head()

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               354107 non-null  int64 
 1   vehicle_type        316623 non-null  object
 2   registration_year   354107 non-null  int64 
 3   gearbox             334277 non-null  object
 4   power               354107 non-null  int64 
 5   model               334406 non-null  object
 6   mileage             354107 non-null  int64 
 7   registration_month  354107 non-null  int64 
 8   fuel_type           321218 non-null  object
 9   brand               354107 non-null  object
 10  not_repaired        282962 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


Al haber una cantidad considerable de datos ausentes, se ha decidido convertirlos en 0, para que igual se puedan considerar los datos que sí están resueltos por cada usuario. 

In [14]:
#Rellené todos los valores ausentes con 0 para que no tuvieran peso en el modelo. 

df = df.fillna(0)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               354107 non-null  int64 
 1   vehicle_type        354107 non-null  object
 2   registration_year   354107 non-null  int64 
 3   gearbox             354107 non-null  object
 4   power               354107 non-null  int64 
 5   model               354107 non-null  object
 6   mileage             354107 non-null  int64 
 7   registration_month  354107 non-null  int64 
 8   fuel_type           354107 non-null  object
 9   brand               354107 non-null  object
 10  not_repaired        354107 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [16]:
# Aplicación del método One Hot para aprovechar las columnas no numéricas.

df_ohe = pd.get_dummies(df, dtype = int, drop_first = True)

print(df_ohe.shape)

(354107, 313)


## Entrenamiento del modelo 

### División de los datos:

In [17]:
df_train, df_test = train_test_split(df_ohe, test_size = 0.4, train_size = 0.6, random_state = 23)

In [18]:
df_test, df_valid = train_test_split(df_test, test_size = 0.5, train_size = 0.5, random_state = 23)

In [19]:
print(df_train.shape)
print()
print(df_test.shape)
print()
print(df_valid.shape)

(212464, 313)

(70821, 313)

(70822, 313)


In [20]:
#Datos para el entrenamiento.
train_features = df_train.drop(['price'], axis=1)
train_target = df_train['price']

#Datos para el test.
test_features = df_test.drop(['price'], axis=1)
test_target = df_test['price']

#Datos para la validación.
valid_features = df_valid.drop(['price'], axis=1)
valid_target = df_valid['price']

### Modelo 1:


In [21]:
best_model = None

best_result = 10000

best_depth = 0

for depth in range(1, 20): 
    model = DecisionTreeRegressor(max_depth = depth, random_state = 23) 
    model.fit(train_features, train_target)
    valid_predictions = model.predict(valid_features) 
    result = mean_squared_error(valid_target, valid_predictions)**0.5
    
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth

print(f"RECM del mejor modelo en el conjunto de validación es (max_depth = {best_depth}): {best_result}")

RECM del mejor modelo en el conjunto de validación es (max_depth = 15): 2056.8783252171147


### Modelo 2



In [22]:
best_error = 10000 

best_est = 0

best_depth = 0

for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestRegressor(random_state = 23, n_estimators = est, max_depth = depth)
        model.fit(train_features, train_target)
        valid_predictions = model.predict(valid_features)
        error = mean_squared_error(valid_target, valid_predictions)**0.5

        if error < best_error:
            best_error = error
            best_est = est
            best_depth = depth

print("RECM del mejor modelo en el conjunto de validación:", best_error, "n_estimators:", best_est, "best_depth:", best_depth)

RECM del mejor modelo en el conjunto de validación: 2027.6264920760798 n_estimators: 50 best_depth: 10


### Modelo 3



In [23]:
model_lr = LinearRegression()

%time
model_lr.fit(train_features, train_target) 

%time
valid_predictions = model_lr.predict(valid_features) 

result_lr = mean_squared_error(valid_target, valid_predictions)**0.5

print()
print("RECM del modelo de regresión lineal:", result_lr)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 7.15 µs
CPU times: user 4 µs, sys: 11 µs, total: 15 µs
Wall time: 9.06 µs

RECM del modelo de regresión lineal: 3166.551412045057


In [24]:
train_data = lgb.Dataset(train_features, label = train_target)

test_data = lgb.Dataset(valid_features, label = valid_target, reference = train_data)

params = {
    "objective": 'regression',
    "boosting": 'gbdt',
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "seed": 23,
    "force_col_wise":True,
}

%time
gbm = lgb.train(params, train_data, num_boost_round = 20, valid_sets = test_data, callbacks = [lgb.early_stopping(stopping_rounds = 5)])

%time
valid_predictions = gbm.predict(test_features, num_iteration = gbm.best_iteration)

rmse_lgb = mean_squared_error(test_target, valid_predictions) ** 0.5

print(f"The RMSE of prediction is: {rmse_lgb}")

CPU times: user 8 µs, sys: 34 µs, total: 42 µs
Wall time: 164 µs
[LightGBM] [Info] Total Bins 962
[LightGBM] [Info] Number of data points in the train set: 212464, number of used features: 294
[LightGBM] [Info] Start training from score 4424.948834
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[20]	valid_0's l2: 7.41094e+06
CPU times: user 14 µs, sys: 3 µs, total: 17 µs
Wall time: 7.15 µs
The RMSE of prediction is: 2725.225938825051


## Análisis de los modelos

In [25]:
#Decision tree regressor:

model_dt = DecisionTreeRegressor(max_depth = 15, random_state = 23) 

%time
model_dt.fit(train_features, train_target)

%time
valid_predictions = model_dt.predict(valid_features) 

result_dt = mean_squared_error(valid_target, valid_predictions)**0.5

print()
print('RECM del modelo Decision Tree Regressor:', result_dt)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs

RECM del modelo Decision Tree Regressor: 2056.8783252171147


In [26]:
#Random forest regressor:

model_rf = RandomForestRegressor(random_state = 23, n_estimators = 50, max_depth = 10)

%time
model_rf.fit(train_features, train_target)

%time
valid_predictions = model_rf.predict(valid_features)

result_rf = mean_squared_error(valid_target, valid_predictions)**0.5

print()
print('RECM del modelo Random Forest Regressor:', result_rf)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs
CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs

RECM del modelo Random Forest Regressor: 2027.6264920760798


In [27]:
results_table = pd.DataFrame({'Model':['Linear Regression','Decision Tree Regressor','Random Forest Regressor',
                                       'LightGBM'],'RECM':[result_lr, result_dt, result_rf, rmse_lgb]})

results_table


,Model,RECM
0,Linear Regression,3166.551412
1,Decision Tree Regressor,2056.878325
2,Random Forest Regressor,2027.626492
3,LightGBM,2725.225939


## Conclusiones: 

En la anterior tabla podemos observar que: 
- Los modelos de Decision Tree Regressor y Random Forest Regressor superaron la prueba de cordura de Linear Regression, inclusive con su potenciación del gradiente efectuada gracias a la librería LightGBM.

- El modelo con menos errores de predicción (un RECM más bajo) fue el Random Forest Regressor, equivocándose en un 0.57 % de las predicciones. Además, sus tiempos de entrenamiento y predicción también son los mas bajos al tardarse tan solo 6.91 µs en el entrenamiento y 3.1 µs en las predicciones. 

- Si bien el modelo de Linear Regression tiene los mejores tiempos de entrenamiento y predicción con 6.2 µs y 2.86 µs respectivamente, este se descarta al ser el modelo con mayor cantidad de errores de predicción (3166.55 equivalente al 0.89 % de posibilidad de error). 

Es por todo lo anterior que se recomienda a Rusty Bargain desarrollar su aplicación con el modelo de Random Forest Regressor, el cual le proporcionará rapidez y exactitud para su público. 

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos